In [1]:
import json
import ast
import random
import seaborn as sns

In [2]:
p_train = r"D:\Documents\Uni\3\Thesis\Data\GettingToKnowYou\train.txt"
p_train_conv = r"D:\Documents\Uni\3\Thesis\Data\GettingToKnowYou\train.json"

p_dev = r"D:\Documents\Uni\3\Thesis\Data\GettingToKnowYou\dev.txt"
p_dev_conv = r"D:\Documents\Uni\3\Thesis\Data\GettingToKnowYou\dev.json"

p_test = r"D:\Documents\Uni\3\Thesis\Data\GettingToKnowYou\test.txt"
p_test_conv = r"D:\Documents\Uni\3\Thesis\Data\GettingToKnowYou\test.json"

# sample paths:
sample_paths = [
    [
        r"D:\Documents\Uni\3\Thesis\Data\GettingToKnowYou\train_k4.json",
        r"D:\Documents\Uni\3\Thesis\Data\GettingToKnowYou\train_k16.json",
        r"D:\Documents\Uni\3\Thesis\Data\GettingToKnowYou\train_k32.json"  
    ],
    [
        r"D:\Documents\Uni\3\Thesis\Data\GettingToKnowYou\dev_k4.json",
        r"D:\Documents\Uni\3\Thesis\Data\GettingToKnowYou\dev_k16.json",
        r"D:\Documents\Uni\3\Thesis\Data\GettingToKnowYou\dev_k32.json"
    ]
]


orig_paths = [p_train, p_dev, p_test]
conv_paths = [p_train_conv, p_dev_conv, p_test_conv]

In [3]:
def create_element(premise, attribute = None):
    element = {}
    element['premise'] = premise
    if attribute == None:
        element['relation'] = 'no_relation'
        element['subj'] = 'dummy'
        element['obj'] = 'dummy'
    else:
        element['relation'] = attribute[1]
        element['subj'] = attribute[0]
        element['obj'] = attribute[2]
    return element

def convert_data(line, counter):
    invalid_relations = ["other", "<blank>"]
    elements = []
    # tab is used as delimiter, so the line can be easily split into the separate parts
    line_split = line.split("\t")
    # first entry is conversation turn, which is not needed.
    # second entry is the actual utterance.
    # third until last entry are the attributes that are annotated.
    # if there are multiple attributes, each is converted into a separate relation instance
    # (with the same premise)
    premise = line_split[1]
    if len(line_split) == 2:
        elements.append(create_element(premise.strip()))
    else:
        attributes = [ast.literal_eval(x.strip()) for x in line_split[2:len(line_split)]]
        for attribute in attributes:
            if attribute[1] not in invalid_relations:
                elements.append(create_element(premise, attribute))
            else:
                counter +=1
    return elements, counter

In [4]:
for i in range(3):
    p_orig = orig_paths[i]
    p_conv = conv_paths[i]
    cleaned_data = []
    with open(p_orig) as file:
        counter = 0
        for line in file:
            if 'persona' not in line:
                line_clean, counter = convert_data(line, counter)
                cleaned_data.extend(line_clean)
        print(counter)
    #with open(p_conv, "wt") as f:
    #    f.write(json.dumps(cleaned_data))

4233
539
525


In [5]:
def load_relations(path):
    relations = {}
    # add each element to corresponding label
    with open(path, "r", encoding="utf-8") as f:
            dataset = json.loads(f.read())
            total_size = len(dataset)
            for entry in dataset:
                rel = entry['relation']
                if rel not in relations.keys():
                    relations[rel] = []
                relations[rel].append(entry)
    return relations

In [9]:
labels = [
        "has_profession",
        "employed_by_general",
        "like_food",
        "physical_attribute",
        "place_origin",
        "like_read",
        "have",
        "have_chidren",
        "like_activity",
        "have_sibling",
        "like_drink",
        "marital_status",
        "live_in_citystatecountry",
        "employed_by_company",
        "teach", "dislike",
        "attend_school",
        "like_music",
        "job_status",
        "favorite_season",
        "like_animal",
        "want_do",
        "have_pet",
        "own",
        "favorite_activity",
        "has_hobby",
        "has_ability",
        "school_status",
        "favorite_color",
        "favorite_music",
        "has_age",
        "misc_attribute",
        "live_in_general",
        "gender",
        "favorite_food",
        "like_general",
        "previous_profession",
        "have_vehicle",
        "like_sports",
        "favorite_drink",
        "favorite_animal",
        "has_degree",
        "like_goto",
        "favorite_music_artist",
        "want",
        "want_job",
        "like_watching",
        "favorite_sport",
        "member_of",
        "have_family",
        "not_have",
        "favorite_show",
        "like_movie",
        "favorite_hobby",
        "favorite",
        "nationality",
        "favorite_movie",
        "favorite_place",
        "favorite_book"
    ]

In [7]:
ks = [4,16,32]
for i in range(2):
    relations = load_relations(conv_paths[i])
    if i ==2:
        print(relations.keys())
    for j in range(3):
        samples = []
        for key in relations.keys():
            if key == 'no_relation':
                continue
            if ks[j] < len(relations[key]):
                samples.extend(random.sample(relations[key], k=ks[j]))
            else:
                samples.extend(relations[key])
        samples.extend(random.sample(relations['no_relation'], k=len(samples)))
        with open(sample_paths[i][j], "wt") as f:
            f.write(json.dumps(samples))

In [10]:
relations = load_relations(p_test_conv)
len(relations.keys())

56

In [14]:
for key in relations.keys():
    if key not in labels:
        print(key)
for label in labels:
    if label not in relations.keys():
        print(label)

no_relation
nationality
favorite_movie
favorite_place
favorite_book


In [12]:
len(labels)

59